In [1]:
import pandas as pd
import httpx
import logging

C:\Users\anaph\AppData\Local\Temp\ipykernel_10524\2104664566.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Configuração de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%d/%m/%Y %H:%M:%S'
)

logger = logging.getLogger(__name__)

def extrair_dados_municipios() -> list:
    """
    Realiza a requisição à API do IBGE para buscar dados sobre os municípios.
    
    Retorna:
        list: Lista de dicionários com os dados dos municípios ou lista vazia em caso de erro.
    """
    url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
    
    try:
        with httpx.Client(timeout=10) as client:
            resposta = client.get(url)
            resposta.raise_for_status()  # Verifica se o status é 200
            return resposta.json() if resposta.status_code == 200 else []
    except httpx.HTTPStatusError as e:
        logger.error(f"Erro HTTP ao buscar dados: {e.response.status_code} - {e.request.url}")
        return []
    except httpx.RequestError as e:
        logger.error(f"Erro durante a requisição: {e}")
        return []
    except Exception as e:
        logger.error(f"Erro inesperado: {e}")
        return []

def criar_tabela_dim_municipios(dados_municipios: list) -> pd.DataFrame:
    """
    Cria um DataFrame com os dados dos municípios e organiza como tabela dim_municipios.
    
    Parâmetros:
        dados_municipios (list): Lista de dicionários com os dados dos municípios.
    
    Retorna:
        pd.DataFrame: DataFrame com os dados organizados.
    """
    if not dados_municipios:
        logger.warning("Nenhum dado de município disponível para processar.")
        return pd.DataFrame()

    # Extraindo os dados de interesse
    municipios = []
    for municipio in dados_municipios:
        try:
            nome = municipio['nome']
            id_municipio = municipio['id']
            microrregiao = municipio['microrregiao']['nome']
            mesorregiao = municipio['microrregiao']['mesorregiao']['nome']
            uf = municipio['microrregiao']['mesorregiao']['UF']['nome']
            regiao = municipio['microrregiao']['mesorregiao']['UF']['regiao']['nome']
        except KeyError as e:
            logger.warning(f"Dados incompletos no município: {municipio}. Erro: {e}")
            continue

        municipios.append({
            'id_municipio': id_municipio,
            'nome': nome,
            'microrregiao': microrregiao,
            'mesorregiao': mesorregiao,
            'uf': uf,
            'regiao': regiao
        })

    if not municipios:
        logger.warning("Nenhum dado foi processado corretamente.")
    
    # Criando o DataFrame
    df_municipios = pd.DataFrame(municipios)
    return df_municipios

def salvar_tabela_excel(df: pd.DataFrame, caminho_arquivo: str):
    """
    Salva o DataFrame em um arquivo Excel.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados a serem salvos.
        caminho_arquivo (str): Caminho completo onde o arquivo Excel será salvo.
    """
    if df.empty:
        logger.warning("O DataFrame está vazio. Nenhum arquivo será salvo.")
        return
    
    try:
        df.to_excel(caminho_arquivo, index=False)
        logger.info(f"Arquivo Excel salvo com sucesso em {caminho_arquivo}")
    except Exception as e:
        logger.error(f"Erro ao salvar o arquivo Excel: {e}")

# Executando as funções
if __name__ == "__main__":
    dados_municipios = extrair_dados_municipios()
    if dados_municipios:
        df_municipios = criar_tabela_dim_municipios(dados_municipios)
        salvar_tabela_excel(df_municipios, r'C:\Users\anaph\OneDrive\Área de Trabalho\DadosMeteorologicos\dim_municipios.xlsx')

08/10/2024 22:15:37 - INFO - HTTP Request: GET https://servicodados.ibge.gov.br/api/v1/localidades/municipios "HTTP/1.1 200 OK"
08/10/2024 22:15:38 - INFO - Arquivo Excel salvo com sucesso em C:\Users\anaph\OneDrive\Área de Trabalho\DadosMeteorologicos\dim_municipios.xlsx
